In [1]:
!pip install streamlit 
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 42.7 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 62.4 MB/s 
     |████████████████████████████████| 235 kB 67.7 MB/s 
     |████████████████████████████████| 164 kB 65.2 MB/s 
     |████████████████████████████████| 181 kB 74.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 9.8 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=2c5149fef1635f78ed2ff6704d5a7f253a15be67c1c91073200178c8c619117a
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 3

In [ ]:
from google.colab import files
files.upload()

In [7]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer


model=pickle.load(open('zstack_model.pkl', 'rb'))
st.title('REVIEW SENTIMENT PREDICTION!!!')
uploaded_file = st.file_uploader("Choose a file")
if st.button("predict"):
  if uploaded_file is not None: 
    count=0
    xls = pd.ExcelFile(uploaded_file)
    geo= pd.read_excel(xls,'geo',index_col="Unnamed: 0")
    item=pd.read_excel(xls,'item',index_col="Unnamed: 0")
    customer= pd.read_excel(xls,'customer',index_col="Unnamed: 0")
    review=pd.read_excel(xls,'review',index_col="Unnamed: 0")
    orders= pd.read_excel(xls,'orders',index_col="Unnamed: 0")
    products=pd.read_excel(xls,'products',index_col="Unnamed: 0")
    sellers= pd.read_excel(xls,'sellers',index_col="Unnamed: 0")
    payment=pd.read_excel(xls,'payment',index_col="Unnamed: 0")

    df=pd.merge(orders,payment,on="order_id",how="inner")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,customer,on="customer_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,item,on="order_id",how="inner")
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=["order_id"],inplace=True)
    df=pd.merge(df,products,on="product_id",how="inner")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,review,on="order_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,sellers,on="seller_id",how="outer")
    df.drop_duplicates(inplace=True)
    df.replace(to_replace="bebes",
              value="babies")
    df.replace(to_replace="ferramentas_jardim",
              value="garden_tools")
    df.replace(to_replace="informatica_acessorios",
              value="informatica_accessories")
    df["review_comment_message"].fillna("nao_reveja",inplace=True)
    df.drop(columns=["review_answer_timestamp","review_creation_date","review_id","review_comment_title","order_delivered_carrier_date"],axis=1,inplace=True)
    df1=df.copy()
    df1["review_score"]=df1["review_score"].apply(lambda x: 0 if x < 3 else 1)
    df1["order_purchase_timestamp"]=pd.to_datetime(df1["order_purchase_timestamp"])
    df1["order_approved_timestamp"]=pd.to_datetime(df1["order_approved_at"])
    df1["order_purchase_date"]=df1.order_purchase_timestamp.dt.date
    df1["approval_date"]=df1["order_approved_timestamp"].dt.date
    df1["order_estimated_delivery_date"]=pd.to_datetime(df1.order_estimated_delivery_date).dt.date
    df1["order_delivered_customer_date"]=pd.to_datetime(df1.order_delivered_customer_date).dt.date
    df1.drop_duplicates(subset=["order_id","customer_id","order_status","order_status","order_approved_at","order_delivered_customer_date","product_weight_g","product_category_name_english","review_comment_message"],inplace=True)
    print("data retained after doing doing data cleaning {:.1f}".format((df1.shape[0]/df.shape[0])*100))
    import nltk
    nltk.download("stopwords")
    from nltk.corpus import stopwords
    from tqdm import tqdm
    from nltk.stem import RSLPStemmer
    import re
    nltk.download("rslp")
    stopwords_portugues=stopwords.words("portuguese")
    stopwords_portugues.remove("nem")
    stopwords_portugues.remove("não")
    stemmer=RSLPStemmer()
    preprocessed_reviews = []
    for sentance in tqdm(df1['review_comment_message'].values):
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance=re.sub("[nN][ãaAÃ][oO][Ññ]|[nN]","Negação",sentance)
        sentance=sentance.replace("\\r",'')
        sentance=sentance.replace("\\n",'')
        sentance = re.sub('[^A-Za-z]+',' ',sentance)
        sentance = ' '.join(a for a in sentance.split() if a.lower not in stopwords_portugues)
        sentance = ' '.join(stemmer.stem(e.lower())for e in sentance.split())
        
        preprocessed_reviews.append(sentance.strip())


    f10=df1.groupby("order_id").count()["product_id"]
    f10.df=pd.DataFrame()
    f10.df["order_id"]=f10.index
    f10.df["F10_number_of_product_ordered"]=f10.values
    f9=df1.groupby("product_id").count()["seller_id"]
    f9.df=pd.DataFrame()
    f9.df["product_id"]=f9.index
    f9.df["number_of_sellers"]=f9.values
    df1["f16_purchase_month"]=(df1["order_purchase_timestamp"]).apply(lambda x: x.month)
    df1["f14_delivery_days"]=(df1["order_delivered_customer_date"]-df1["order_purchase_date"]).dt.days
    df1=pd.merge(df1,f9.df,on="product_id")
    df1=pd.merge(df1,f10.df,on="order_id")

    df1["f1_delivery_estimated_days"]=(df1["approval_date"]-df1["order_estimated_delivery_date"]).dt.days
    df1["f2_delivery_actual_days"]=(df1["approval_date"]-df1["order_delivered_customer_date"]).dt.days
    df1["f3_on_time_or_late"]=(df1["f1_delivery_estimated_days"] > df1["f2_delivery_actual_days"]).astype("int")
    df1["f4_discount_or_not"]=(df1["payment_value"]-(df1["price"]+df1["freight_value"])).apply(lambda x: 1 if x < 0 else 0)#converted to boolean 
    df1["f5_purchase_approved_delay"]=(df1["approval_date"]-df1["order_purchase_date"]).dt.days
    df1["f6_Average_product_value_per_order"]=df1.price/(df1.F10_number_of_product_ordered)
    df1["f7_order_frieght_value"]=df1.freight_value/df1.price
    df1["f8_purchase_day_week"]=df1.order_purchase_timestamp.dt.dayofweek
    df1["f11_order_purch_day"]=df1.order_purchase_timestamp.dt.day
    df1["f12_words_per_review"]=df1.review_comment_message.apply(lambda x: len(x.split(" ")))
    df1["f13_order_purchase_hour"]=df1["order_purchase_timestamp"].apply(lambda x: x.hour)
    df1["f15_week_delivery"]=df1["order_purchase_timestamp"].apply(lambda x: x.dayofweek)


    df1["review_comment_message"]=preprocessed_reviews


    
    xtest=df1.copy()
    xtest.drop(['order_id'],axis=1,inplace=True)
    xtest=xtest[["product_description_lenght","product_name_lenght","order_status","payment_type","product_category_name_english","payment_installments","product_weight_g","price","payment_sequential","product_photos_qty","f13_order_purchase_hour","f6_Average_product_value_per_order","payment_value","f11_order_purch_day","f12_words_per_review",'f2_delivery_actual_days','f3_on_time_or_late', 'f4_discount_or_not','F10_number_of_product_ordered','review_score','review_comment_message']]
    xtedon=xtest[['f3_on_time_or_late', 'f4_discount_or_not']]
    
    if 0<xtest['f6_Average_product_value_per_order'].values<7000:
        norm=pickle.load(open('znorm1.pkl', 'rb'))
        xte_apv=norm.transform(xtest['f6_Average_product_value_per_order'].values.reshape(-1,1))
        count+=1
    else:
        st.write("enter correct value of average product value per order between 0-7000")
        xte_apv=0
    
    
    from sklearn.feature_extraction.text import CountVectorizer
    
    if 3 < xtest["product_description_lenght"].values < 4000:
        norm=pickle.load(open('znorm3.pkl', 'rb'))
        xte_pdl=norm.transform(xtest["product_description_lenght"].values.reshape(-1,1))
        count+=1
    else:
         st.write("enter correct product description length between 3-4000")
         xte_pdl=0
        
    if 4 < xtest["product_name_lenght"].values < 80:
        norm=pickle.load(open('znorm4.pkl', 'rb'))
        xte_pnl=norm.transform(xtest["product_name_lenght"].values.reshape(-1,1))
        count+=1
    
    else:
      st.write("enter correct product name length between 4-80")
      xte_pnl=0   
   
    if 0 < xtest["payment_value"].values < 14000:
        norm=pickle.load(open('znorm5.pkl', 'rb'))
        xte_pv=norm.transform(xtest["payment_value"].values.reshape(-1,1))
        count+=1
    else:
        st.write("enter valid payment value between 0-14000")
        xte_pv=0
    
    if 0 < xtest["price"].values < 7000:
          norm=pickle.load(open('znorm7.pkl', 'rb'))
          xte_pri=norm.transform(xtest['price'].values.reshape(-1,1))
          count+=1
    else:
       st.write("enter the correct price between 0-7000")
       xte_pri=0
    

    if 0 < xtest['product_weight_g'].values < 41000:

      norm=pickle.load(open('znorm8.pkl', 'rb'))
      xte_pw=norm.transform(xtest['product_weight_g'].values.reshape(-1,1))
      count+=1
    
    else:
       st.write("enter correct product weight between 0-41000")
       xte_pw=0  
    
    if 0<=xtest['payment_installments'].values<=30:
      norm=pickle.load(open('znorm9.pkl', 'rb'))
      xte_pi=norm.transform(xtest['payment_installments'].values.reshape(-1,1))
      count+=1
    else:
      print("correct payment insatllments 0-30")
      xte_pi=0

    #here we are normalising the numerical features as so that all come to same scale
    norm=pickle.load(open('znorm10.pkl', 'rb'))
    xte_dad=norm.transform(xtest["f2_delivery_actual_days"].values.reshape(-1,1))
     

    norm=pickle.load(open('norm11.pkl', 'rb'))
    xte_opd=norm.transform(xtest["f11_order_purch_day"].values.reshape(-1,1))
    
    if 0< xtest["f12_words_per_review"].values <60:
      norm=pickle.load(open('znorm12.pkl', 'rb'))
      xte_wpr=norm.transform(xtest["f12_words_per_review"].values.reshape(-1,1))
      count+=1
    else:
      st.write("enter limited words in review comment message between  0-60 ")
      xte_wpr=0
    
    
    if 0< xtest['F10_number_of_product_ordered'].values < 10:

      norm=pickle.load(open('znorm13.pkl', 'rb'))
      xte_po=norm.transform(xtest['F10_number_of_product_ordered'].values.reshape(-1,1))
      count+=1
    else:
      st.write("enter correct number of products ordered between 0-10 ")
      xte_po=0


    vect=pickle.load(open('zvect.pkl', 'rb'))
    xte_os=vect.transform(xtest["order_status"].values)



    vect=pickle.load(open('zvect1.pkl', 'rb'))
    xte_pty=vect.transform(xtest["payment_type"].values)


    vect=pickle.load(open('zvect2.pkl', 'rb'))
    xte_pcm=vect.transform(xtest["product_category_name_english"].values)

    norm=pickle.load(open('znorm2.pkl', 'rb'))
    xte_psq=norm.transform(xtest['payment_sequential'].values.reshape(-1,1))
    
    norm=pickle.load(open('znorm6.pkl', 'rb'))
    xte_phr=norm.transform(xtest['f13_order_purchase_hour'].values.reshape(-1,1))
    

    
    
    #here we are vectorising the text data using tfidf such that we are ignoring the terms which are appearing less than 3 times
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer =pickle.load(open('zvectorizer.pkl', 'rb')) 
    xte_rcm=vectorizer.transform(xtest["review_comment_message"].values)
    
    #we are stacking the horizontally the all the vectors formed
    from scipy.sparse import hstack
    xte=hstack((xte_pcm,xte_pty,xte_os,xte_pnl,xte_pdl,xte_apv,xte_psq,xte_pv,xte_phr,xte_pri,xte_pw,xte_pi,xte_dad,xte_wpr,xte_opd,xtedon,xte_po,xte_rcm)).tocsr()
    
    if count==9:
      val=model.predict(xte)
      if val==1:
        st.write("positive review given")
      if val==0:
        st.write("negative review given")




Overwriting app.py


In [4]:
from pyngrok import ngrok
ngrok.set_auth_token('2DQlpVpnhSIFXKSBhEU0y6h4Qw6_4UAwqZpWmhh5EbLzqudSz')

In [8]:
ngrok.kill()

In [9]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://b4c9-34-86-179-206.ngrok.io" -> "http://localhost:80"
